In [62]:
import random
import time
import pandas as pd

## Получение времени выполнения

In [63]:
def get_seconds(func, x, logging = False):
    start = time.time()
    res = func(x)
    if logging:
        print(res)
    end = time.time()
    return end - start

## Свап элементов в массиве (листе)

In [64]:
def swap(idx1: int, idx2: int, mas: list) -> list:
    mas_i: int = mas[idx1]
    mas[idx1] = mas[idx2]
    mas[idx2] = mas_i
    return mas

# Пузырьковая сортировка 

## Обычный вариант

In [65]:
def bubble_sort(mas: list) -> list:
    while(True):
        is_replaced: bool = False
        for i in range(len(mas)):
            if i == len(mas) - 1:
                break

            if mas[i] > mas[i+1]:
                mas = swap(i, i+1, mas)
                is_replaced = True
            
        if not is_replaced:
            break

    return mas

## Шейкерная сортировка

In [66]:
def shaker_sort(mas: list) -> list:
    while(True):
        is_replaced: bool = False
        for i in range(len(mas)):
            if i == len(mas) - 1:
                break

            if mas[i] > mas[i+1]:
                mas = swap(i, i+1, mas)
                is_replaced = True

        for i in range(len(mas)):
            if i == 0:
                break

            if mas[-i] < mas[-i-1]:
                mas = swap(-i, -i-1, mas)
                is_replaced = True

        if not is_replaced:
            break

    return mas

## Сортировка "расческой"

In [67]:
def comb_sort(mas: list) -> list:
    constant: float = 1.247
    step: int = len(mas) - 1
    while(step >= 1):
        for i in range(len(mas)):
            if i == len(mas) - 1 or i+step>len(mas)-1:
                break

            if mas[i] > mas[i+step]:
                mas = swap(i, i+step, mas)

        step = int(step//constant)

    mas = bubble_sort(mas)

    return mas

# Простые сортировки

## Сортировка вставками

In [68]:
def insert_sort(mas: list) -> list:
    for i in range(len(mas)):
        j: int = i
        while((j > 0) & (mas[j-1] > mas[j])):
            mas = swap(j, j-1, mas)
            j -= 1 
        
    return mas

## Сортировка выбором

In [69]:
def choose_sort(mas: list) -> list:
    n: int = len(mas)
    for i in range(n):
        local_min: int = 1e9
        local_min_idx: int = None
        
        for j in range(i+1, n):
            if mas[j] < local_min:
                local_min = mas[j]
                local_min_idx = j
                
        if (local_min_idx is not None) & (local_min < mas[i]):
            mas = swap(i, local_min_idx, mas)
            
    return mas

# Эффективные сортировки

## Быстрая сортировка

In [70]:
def split_mas(mas: list) -> int:
    return random.choice(mas)

In [71]:
def quick_sort(mas: list) -> list:
    if len(mas) < 2:
        return mas
    
    base: list = split_mas(mas)
    less: list = [i for i in mas if i < base]
    more: list = [i for i in mas if i > base]

    return quick_sort(less) + [base] + quick_sort(more)

## Пирамидальная сортировка

In [72]:
def heapify(mas: list, n: int, root: int):
    largest = root
    left = 2*root + 1
    right = 2*root + 2

    if left < n and mas[root] < mas[left]:
        largest = left

    if right < n and mas[largest] < mas[right]:
        largest = right

    if largest != root:
        mas = swap(root, largest, mas)
        heapify(mas, n, largest)

In [73]:
def heap_sort(mas: list) -> list:
    n = len(mas)
    for i in range(n//2 - 1, -1, -1):
        heapify(mas, n, i)
    for i in range(n-1, 0, -1):
        mas = swap(0, i, mas)
        heapify(mas, i, 0) 
        
    return mas

## Сортировка слиянием

In [74]:
def merge(f_half: list, s_half: list) -> list:
    merged: list = []
    while((len(f_half) > 0) & (len(s_half) > 0)):
        if f_half[0] > s_half[0]:
            merged.append(s_half.pop(0))
            
        else:
            merged.append(f_half.pop(0))
            
    return merged + f_half + s_half

In [75]:
def merge_sort(mas: list) -> list:
    n: int = len(mas)

    if n < 2:
        return mas
        
    f_half: list = mas[:n//2]
    s_half: list = mas[n//2:]
    return merge(merge_sort(f_half), merge_sort(s_half))

## *Вывод результатов*

### Функция для проведения n_iters экспериментов размерности size с амплитудой величины элемента amplitude

In [76]:
def show_results(amplitude: int, size: int, funcs: list, names: list, n_iters: int = 1) -> pd.DataFrame:  
    times: dict = dict(zip(names, [0 for i in range(len(names))]))
                       
    for i in range(n_iters):
        mas = [random.randint(-amplitude, amplitude) for i in range(size+1)]          
        for func, name in zip(funcs, names):
            times[name] += get_seconds(
                func,
                mas.copy(),
                False
            )

    df = pd.DataFrame(
        list(times.items()), 
        columns = ['Algorithm', 'Total time']
    )
    
    df.sort_values(
        by = 'Total time', 
        ascending = True, 
        inplace = True
    )

    df['Total time'] = df['Total time'].apply(lambda x: x/n_iters)
    return df

In [77]:
current_funcs = [bubble_sort, shaker_sort, comb_sort, insert_sort, choose_sort, quick_sort, heap_sort, merge_sort]
current_names = ["bubble_original", "bublee_shaker", "bubble_comb", "insert", "choose", "quick", "heap", "merge"]

### Размерность 3 порядка

In [78]:
show_results(
    amplitude = 1000,
    size = 1000,
    n_iters = 50, 
    funcs = current_funcs,
    names = current_names
)

,Algorithm,Total time
5,quick,0.001136
6,heap,0.002612
7,merge,0.002677
2,bubble_comb,0.004070
4,choose,0.013386
3,insert,0.040870
1,bublee_shaker,0.120476
0,bubble_original,0.122889


### Размерность 4 порядка

In [79]:
show_results(
    amplitude = 1000,
    size = 10000,
    n_iters = 5, 
    funcs = current_funcs,
    names = current_names
)

,Algorithm,Total time
5,quick,0.008609
6,heap,0.036527
2,bubble_comb,0.058982
7,merge,0.114608
4,choose,1.336042
3,insert,4.186340
1,bublee_shaker,12.494069
0,bubble_original,12.740224


## Графики

In [80]:
#dimensions = list(range(10, 5000, 10))
#problems = [[random.randint(-1000, 1000) for i in range(dim)] for dim in dimensions]

In [81]:
#import matplotlib.pyplot as plt

#times: dict = {
    #"bubble_original": [get_seconds(bubble_sort, problem.copy(), False) for problem in problems],
    #"bublee_shaker" : [get_seconds(shaker_sort, problem.copy(), False) for problem in problems],
    #"bubble_comb": [get_seconds(comb_sort, problem.copy(), False) for problem in problems],
    #"insert" : [get_seconds(insert_sort, problem.copy(), False) for problem in problems],
    #"choose" : [get_seconds(choose_sort, problem.copy(), False) for problem in problems],
    #"quick" : [get_seconds(quick_sort, problem.copy(), False) for problem in problems],
    #"heap" : [get_seconds(heap_sort, problem.copy(), False) for problem in problems]
#}

#plt.title("$O(f(n))$ visualisation")
#plt.ylabel('time', fontsize= 14 )
#plt.xlabel('dimension', fontsize= 14 )

#for key, value in times.items():
    #plt.plot(dimensions, value, label = key)
    
#plt.legend()